In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle

from Network import Network
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from GridSearch import GridSearcher
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from activation_functions import sigmoid, relu,relu_derivative, sigmoid_derivative, tanh, tanh_derivative

In [2]:
data=pd.read_csv('spam_final_df.data')
data.head()

,word_freq_make,word_freq_address,word_freq_all,word_freq_3d,word_freq_our,word_freq_over,word_freq_remove,word_freq_internet,word_freq_order,word_freq_mail,...,char_freq_;,char_freq_(,char_freq_[,char_freq_!,char_freq_$,char_freq_#,capital_run_length_average,capital_run_length_longest,capital_run_length_total,spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


In [3]:
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=26)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
learning_rate=0.1

In [7]:
# network = Network(learning_rate, epochs=1000, node_counts = [3, 3, 3])

In [ ]:
# params = {
#     "learning_rate": [0.1, 0.2, 0.5],
#     "epochs": [10, 100],
#     "node_counts": [[1], [2], [3], [3, 3], [3, 3, 3]],
# }
# gs = GridSearcher(network, params, X_train, y_train)
# gs_results = gs.search()
# gs_results

In [4]:
network = Network(learning_rate, epochs=1000, node_counts = [3, 3, 3],activation_function=(tanh, tanh_derivative))

In [10]:
params = {
    "learning_rate": [0.1],
    "epochs": [10],
    "node_counts": [[1] ,[3]],
    "activation_function":[(sigmoid,sigmoid_derivative)]
}
gs = GridSearcher(network, params, X_train, y_train)
gs_results = gs.search()
gs_results

1 / 2
2 / 2


,params,test_score,fit_time,score_time,score_mean
0,"{'learning_rate': 0.1, 'epochs': 10, 'node_cou...","[nan, nan, nan, nan, nan]","[4.717294931411743, 4.8212339878082275, 4.6843...","[0.03298187255859375, 0.029984235763549805, 0....",NaN
1,"{'learning_rate': 0.1, 'epochs': 10, 'node_cou...","[nan, nan, nan, nan, nan]","[4.059672594070435, 4.019693374633789, 4.08266...","[0.03098464012145996, 0.032981157302856445, 0....",NaN


In [14]:
# params=gs_results.iloc[gs_results['score_mean'].idxmax()]
# comment/delete the below line when grid search works with valid output
params=gs_results.iloc[0] 

selected_prams=params["params"]
selected_prams

{'learning_rate': 0.1,
 'epochs': 10,
 'node_counts': [1],
 'activation_function': (<function activation_functions.sigmoid(x)>,
  <function activation_functions.sigmoid_derivative(x)>)}

In [16]:
new_network= Network(learning_rate=selected_prams["learning_rate"], epochs=selected_prams["epochs"], node_counts=selected_prams["node_counts"],activation_function=selected_prams["activation_function"]
)

In [6]:
# selected_prams=gs_results.nlargest(1,['score_mean'])
# selected_prams
# a=selected_prams["activation_function"]

# activation=[fun for fun in a.iloc[0]]
# activation=tuple(activation)
# print(activation)

(<function sigmoid at 0x0000018974C89260>, <function sigmoid_derivative at 0x0000018974C89300>)


In [ ]:
new_network.fit(X_train, y_train)

out = new_network.predict(X_test)

In [ ]:
y_pred=np.where(np.array(out) > 0.5, 1, 0)

In [ ]:
def score(y_true, y_pred):
    correct_predictions = 0
    total_predictions = len(y_true)

    for true_label, predicted_label in zip(y_true, y_pred):
        if true_label == predicted_label:
            correct_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy
accuracy=score(y_test,y_pred)
print("accuracy")
print(accuracy)

In [ ]:
print(classification_report(y_pred=np.squeeze(y_pred),y_true=np.squeeze(y_test)))

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.2, random_state=26)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
def create_model(hidden_units, dropout_rate):
    model = Sequential([
        tf.keras.layers.Flatten(input_shape=(57,)),
        Dense(hidden_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(2, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model
model = KerasClassifier(build_fn=create_model)

param_grid = {
    'hidden_units': [64, 128, 256],
    'dropout_rate': [0.2, 0.3, 0.4],
    'batch_size': [32, 64],
    'epochs': [100]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)

# Train the model using cross-validation
grid_search_result = grid_search.fit(X_train, y_train)

In [14]:
with open("grid_search_spam.pickle", "wb") as pickle_file:
    pickle.dump(grid_search_result, pickle_file)